# 如何使用部分提示模板

提示模板是一个具有`.format`方法的类，它接受一个键值映射并返回一个字符串（即提示），以传递给语言模型。与其他方法一样，将提示模板“部分”化可能是有意义的-例如，只传递所需值的子集，以创建一种仅接受其余值子集的新提示模板。

LangChain以两种方式支持此功能：我们允许部分格式化提示（1）使用字符串值，（2）使用返回字符串值的功能。这两种不同的方式支持不同的用例。在下面的文档中，我们将介绍两种用例的动机以及如何在LangChain中执行它。

## 使用字符串进行部分化

要部分化提示模板的常见用例之一是如果您先获取某些变量，然后获取其他变量。例如，假设您有一个提示模板，需要两个变量`foo`和`baz`。如果您在链的早期就获得了`foo`值，而稍后获得了`baz`值，等待在同一位置获取两个变量并将它们传递给提示模板可能很麻烦。相反，您可以使用`foo`值部分化提示模板，然后将部分化提示模板传递并仅使用该模板。以下是执行此操作的示例：

In [1]:
from langchain.prompts import PromptTemplate

In [2]:
prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])
partial_prompt = prompt.partial(foo="foo");
print(partial_prompt.format(bar="baz"))

foobaz


您还可以使用部分化变量来初始化提示。

In [3]:
prompt = PromptTemplate(template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"})
print(prompt.format(bar="baz"))

foobaz


## 使用函数进行部分化

另一个常见的用例是使用函数进行部分化。这种情况下的用例是您知道您始终希望以常见方式获取某个变量。这种情况的主要示例是日期或时间。想象一下，您有一个提示，始终希望具有当前日期。您不能将其硬编码到提示中，并且使用其他输入变量传递它有点麻烦。在这种情况下，能够使用一个始终返回当前日期的函数对提示进行部分化非常方便。

In [4]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [5]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", 
    input_variables=["adjective", "date"]
);
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 02/27/2023, 22:15:16


您还可以使用部分化变量来初始化提示，这在这个工作流程中经常更有意义。

In [6]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", 
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime}
);
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 02/27/2023, 22:15:16
